<a href="https://colab.research.google.com/github/yesoly/MachineLearningProject/blob/master/Assignment_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classification for Multiple Categories using Pytorch
Build a classifier for the digit classification task with 10 classes on the MNIST dataset

In [ ]:
import os

# load data
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# train
import torch
from torch import nn
from torch.nn import functional as F
from torch.autograd import Variable
import numpy as np

# visualization
import matplotlib.pyplot as plt

## 1. Data

- apply normalization

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,)),  # mean value = 0.1307, standard deviation value = 0.3081
])

- load the MNIST dataset

In [ ]:
data_path = './MNIST'

training_set = datasets.MNIST(root = data_path, train= True, download=True, transform= transform)
testing_set = datasets.MNIST(root = data_path, train= False, download=True, transform= transform)

## 2. Model
- design a neural network that consists of three fully connected layers with an activation function of Sigmoid
- the activation function for the output layer is LogSoftmax

In [ ]:
class classification(nn.Module):
    def __init__(self):
        super(classification, self).__init__()
        
        # construct layers for a neural network
        self.classifier1 = nn.Sequential(
            nn.Linear(in_features=28*28, out_features=20*20),
            nn.Sigmoid(),
        ) 
        self.classifier2 = nn.Sequential(
            nn.Linear(in_features=20*20, out_features=10*10),
            nn.Sigmoid(),
        ) 
        self.classifier3 = nn.Sequential(
            nn.Linear(in_features=10*10, out_features=10),
            nn.LogSoftmax(dim=1),
        ) 
        
        
    def forward(self, inputs):                 # [batchSize, 1, 28, 28]
        x = inputs.view(inputs.size(0), -1)    # [batchSize, 28*28]
        x = self.classifier1(x)                # [batchSize, 20*20]
        x = self.classifier2(x)                # [batchSize, 10*10]
        out = self.classifier3(x)              # [batchSize, 10]
        
        return out

## 3. Loss function

- the log of softmax
- the negative log likelihood loss

In [ ]:
criterion = nn.NLLLoss()

## 4. Optimization

- use a stochastic gradient descent algorithm with different mini-batch sizes of 32, 64, 128
- use a constant learning rate for all the mini-batch sizes
- do not use any regularization algorithm such as dropout or weight decay
- compute the average loss and the average accuracy for all the mini-batches within each epoch

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
classifier = classification().to(device)
learning_rate_value = 1e-4
optimizer = torch.optim.SGD(classifier.parameters(), lr=learning_rate_value)

## 6. Train

In [ ]:
def train(model, batch_size, optimizer, criterion):
    model.train()
    train_loss_list, train_acc_list = [], []
    train_accuracy = 0.0
    train_loader = torch.utils.data.DataLoader(dataset=training_set, batch_size=batch_size, shuffle=True)
    for batch_idx, (train_img, train_label) in enumerate(train_loader):
        train_img, train_label = train_img.to(device), train_label.to(device)
        train_output = model(train_img)
        loss  = criterion(train_output, train_label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss
        train_pred = train_output.argmax(dim=1, keepdim=True)
        train_accuracy += train_pred.eq(train_label.view_as(train_pred)).sum().item()
        
    print("Training Loss: {:.4f} ".format(train_loss/len(train_loader)),
          "Train Accuracy: {:.4f}".format(train_accuracy/len(train_loader.dataset)))
    
    return train_loss_list, train_acc_list


def test(model, batch_size, optimizer, criterion):
    model.eval()
    test_loss_list, test_acc_list = [], []
    test_loss = 0.0
    test_accuracy = 0.0
    test_loader = torch.utils.data.DataLoader(dataset=testing_set, batch_size=batch_size, shuffle=False)
    with torch.no_grad():
        for test_img, test_label in test_loader:
            test_img, test_label = test_img.to(device), test_label.to(device)
            test_output = model(test_img)
            test_loss += criterion(test_output.data, test_label)
            test_pred = test_output.argmax(dim=1, keepdim=True)
            test_accuracy += test_pred.eq(test_label.view_as(test_pred)).sum().item()

    print("Test Loss: {:.4f} ".format(test_loss/len(test_loader)),
          "Test Accuracy: {:.4f}".format(test_accuracy / len(test_loader.dataset)))
    
    return test_loss_list, test_acc_list

In [ ]:
def run_epoch(model, batch_size, optimizer, criterion):
    train_loss_list, train_acc_list, test_loss_list, test_acc_list = [], [], [], []
    for epoch in range(epochs):
        print("Epoch: {}/{} : ".format(epoch+1, epochs))
        train_loss, train_acc = train(model, batch_size, optimizer, criterion)
        test_loss, test_acc = test(model, batch_size, optimizer, criterion)
        
        train_loss_list.append(train_loss)
        train_acc_list.append(train_acc)
        test_loss_list.append(test_loss)
        test_acc_list.append(test_acc)
    
    return train_loss_list, test_loss_list, train_acc_list, test_acc_list

In [ ]:
train_loss, test_loss, train_acc, test_acc = run_epoch(classifier, 32, optimizer, criterion)

In [ ]:
            train_loss_list.append(training_loss/len(train_loader))
            test_loss_list.append(testing_loss/len(test_loader))
            train_acc_list.append(train_accuracy/len(train_loader))
            test_acc_list.append(test_accuracy/len(test_loader))

In [ ]:
def training_mnist (model, batch_size, optimizer, criterion):
    train_loss_list, train_acc_list, test_loss_list, test_acc_list = [], [], [], []
    train_loader = torch.utils.data.DataLoader(dataset=training_set, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(dataset=testing_set, batch_size=batch_size, shuffle=False)
    print("total train batch num = ",len(train_loader))
    for epoch in range(epochs):
        training_loss = 0.0
        train_accuracy = 0.0

        for train_img, train_label in train_loader:
            train_img, train_label = train_img.to(device), train_label.to(device)
            train_output = model(train_img)
            loss  = criterion(train_output, train_label)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    
            training_loss += loss / len(train_loader)
            train_pred = train_output.argmax(dim=1, keepdim=True)
            train_accuracy += train_pred.eq(train_label.view_as(train_pred)).sum().item()

        else:
            testing_loss = 0.0
            test_accuracy = 0.0

            with torch.no_grad():
                test_img = test_loader.data.view(-1, 28 * 28).float().to(device)
                test_label = test_loader.targets.to(device)
                test_output = model(test_img)
                    
                testing_loss += criterion(test_output.data, test_label)
                test_pred = torch.argmax(test_output, 1) == test_label
                test_accuracy = test_pred.float().mean()

            train_loss_list.append(training_loss/len(train_loader))
            test_loss_list.append(testing_loss/len(test_loader))
            train_acc_list.append(train_accuracy/len(train_loader))
            test_acc_list.append(test_accuracy/len(test_loader))

            print("Epoch: {}/{} ==> ".format(epoch+1, epochs),
                  "Training Loss: {:.4f} ".format(training_loss/len(train_loader.dataset)),
                  "Train Accuracy: {:.4f}".format(train_accuracy/len(test_loader.dataset)),
                  "Test Loss: {:.4f} ".format(testing_loss/len(test_loader.dataset)),
                  "Test Accuracy: {:.4f}".format(test_accuracy.item()))

    return train_loss_list, test_loss_list, train_acc_list, test_acc_list

In [ ]:
epochs = 50

In [ ]:
# mini-batch size = 32
train_loss, test_loss, train_acc, test_acc = training_mnist (classifier, 32, optimizer, criterion)

In [ ]:
# mini-batch size = 64
train_loss, test_loss, train_acc, test_acc = training_mnist (classifier, 64, optimizer, criterion)

# mini-batch size = 128
train_loss, test_loss, train_acc, test_acc = training_mnist (classifier, 128, optimizer, criterion)

## 5. Output

1. Plot the training and testing losses with a batch size of 32 [4pt]

2. Plot the training and testing accuracies with a batch size of 32 [4pt]

3. Plot the training and testing losses with a batch size of 64 [4pt]

4. Plot the training and testing accuracies with a batch size of 64 [4pt]

5. Plot the training and testing losses with a batch size of 128 [4pt]

6. Plot the training and testing accuracies with a batch size of 128 [4pt]

7. Print the loss at convergence with different mini-batch sizes [3pt]

8. Print the accuracy at convergence with different mini-batch sizes [3pt]